"FC layers referenced from https://towardsdatascience.com/math-neural-network-from-scratch-in-python-d6da9f29ce65"


In [1]:
import torch, time, os, pickle
import numpy as np
import torch.nn as nn
import pandas as pd
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, Dataset
import torch.optim as optim
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_fscore_support
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest, f_classif, chi2
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objs as go
from sklearn.manifold import TSNE
import plotly.io as pio
from sklearn.utils import class_weight
import tqdm as notebook_tqdm
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tqdm import tqdm
from graph_context_dataset import FeatureEngineeredDataset
import warnings
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense, Dropout
import random
from model import FCClassifier, MyNetwork, DATASET_PATH, MatchingAttention
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import chi2
from sklearn.metrics import f1_score, confusion_matrix, classification_report
%load_ext autoreload
%autoreload 2

C:\Users\edayo\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Make sure to specify which dataset to use

 - dataset_original
 - dataset_drop_noise
 - dataset_smote

In [2]:
# dataset_path = "dataset_original"
# dataset_path = "dataset_drop_noise"
# dataset_path = "dataset_smote"
dataset_path = DATASET_PATH

<h3> Declare functions

In [3]:
# class FCLayer(nn.Module):
#     def __init__(self, input_dim, output_dim):
#         super(FCLayer, self).__init__()
#         self.fc = nn.Linear(input_dim, output_dim)

#     def forward(self, x):
#         x = self.fc(x)
#         return x

# class ActivationLayer(nn.Module):
#     def __init__(self, activation_fn):
#         super(ActivationLayer, self).__init__()
#         self.activation_fn = activation_fn

#     def forward(self, x):
#         x = self.activation_fn(x)
#         return x

# def tanh(x):
#     return torch.tanh(x)

# def sigmoid(x):
#     return torch.sigmoid(x)
# # loss function and its derivative
# def mse(y_true, y_pred):
#     return np.mean(np.power(y_true - y_pred, 2))

# def mse_prime(y_true, y_pred):
#     return 2 * (y_pred - y_true) / y_true.size


In [4]:
def oversample_data(X_train, Y_train, num_classes):
    # Determine the class with the maximum number of instances
    max_class_count = np.max(np.bincount(Y_train))
    # Generate indices for oversampling each class
    indices_list = [np.where(Y_train == i)[0] for i in range(num_classes)]
    # Oversample minority classes to match the count of the majority class
    for i, indices in enumerate(indices_list):
        if len(indices) < max_class_count:
            # Calculate the number of instances to oversample for this class
            num_to_oversample = max_class_count - len(indices)
            # Randomly select instances with replacement to oversample
            oversampled_indices = np.random.choice(indices, size=num_to_oversample, replace=True)
            # Append the oversampled instances to the original data
            X_train = np.concatenate((X_train, X_train[oversampled_indices]), axis=0)
            Y_train = np.concatenate((Y_train, Y_train[oversampled_indices]), axis=0)
    return torch.tensor(X_train), torch.tensor(Y_train)


In [5]:
def concatenate_tensors(tensor_list):
    if not tensor_list:
        raise ValueError("The tensor list is empty")

    feature_dim = tensor_list[0].shape[1]
    for tensor in tensor_list:
        if tensor.shape[1] != feature_dim:
            raise ValueError("All tensors must have the same feature dimension")
    
    concatenated_tensor = torch.cat(tensor_list, dim=0)
    
    return concatenated_tensor

<h4> Import labels and label decoder

In [6]:
file_path = "data/dump/" + dataset_path + "/labels_train.pkl"
with open(file_path, 'rb') as file:
    y_train = pickle.load(file)
y_train = torch.tensor(y_train)

file_path = "data/dump/" + dataset_path + "/labels_test.pkl"
with open(file_path, 'rb') as file:
    y_test = pickle.load(file)
y_test = torch.tensor(y_test)

file_path = "data/dump/" + dataset_path + "/labels_dev.pkl"
with open(file_path, 'rb') as file:
    y_val = pickle.load(file)
y_val = torch.tensor(y_val)
    
file_path = 'data/dump/' + dataset_path + '/label_decoder.pkl'
with open(file_path, 'rb') as file:
    label_decoder = pickle.load(file)

<h4> Import the BERT base-node outputs

first we disregard the u' and directly train the h'

In [7]:
train_file_paths = [
    "embed/" + dataset_path + "/u_prime_BERT_train.pkl",
    "embed/" + dataset_path + "/h_prime_BERT_DGCN_train.pkl",
    "embed/" + dataset_path + "/h_prime_BERT_GATv1_train.pkl",
    "embed/" + dataset_path + "/h_prime_BERT_GATv1_edgeAttr_train.pkl",
    "embed/" + dataset_path + "/h_prime_BERT_GATv2_edgeAttr_train.pkl",
    "embed/" + dataset_path + "/h_prime_BERT_RGAT_train.pkl",
    "embed/" + dataset_path + "/h_prime_BERT_EGAT_train.pkl",
]

test_file_paths = [
    "embed/" + dataset_path + "/u_prime_BERT_test.pkl",
    "embed/" + dataset_path + "/h_prime_BERT_DGCN_test.pkl",
    "embed/" + dataset_path + "/h_prime_BERT_GATv1_test.pkl",
    "embed/" + dataset_path + "/h_prime_BERT_GATv1_edgeAttr_test.pkl",
    "embed/" + dataset_path + "/h_prime_BERT_GATv2_edgeAttr_test.pkl",
    "embed/" + dataset_path + "/h_prime_BERT_RGAT_test.pkl",
    "embed/" + dataset_path + "/h_prime_BERT_EGAT_test.pkl",
]

val_file_paths = [
    "embed/" + dataset_path + "/u_prime_BERT_dev.pkl",
    "embed/" + dataset_path + "/h_prime_BERT_DGCN_dev.pkl",
    "embed/" + dataset_path + "/h_prime_BERT_GATv1_dev.pkl",
    "embed/" + dataset_path + "/h_prime_BERT_GATv1_edgeAttr_dev.pkl",
    "embed/" + dataset_path + "/h_prime_BERT_GATv2_edgeAttr_dev.pkl",
    "embed/" + dataset_path + "/h_prime_BERT_RGAT_dev.pkl",
    "embed/" + dataset_path + "/h_prime_BERT_EGAT_dev.pkl",
]

dictKey = {
    0 : 'bert',
    1 : 'bert-select-few',
    2 : 'bert-select-mod',
    3 : 'bert-select-more',
    4 : 'dgcn',
    5 : 'dgcn-select',
    6 : 'gatv1',
    7 : 'gatv1-select',
    8 : 'gatv1-edge',
    9 : 'gatv1-edge-select',
    10 : 'gatv2-edge',
    11 : 'gatv2-edge-select',
    12 : 'rgat',
    13 : 'rgat-select',
    14 : 'egat',
    15 : 'egat-select',
    16 : 'bert-select-mod-dgcn',
    17 : 'bert-select-mod-gatv1',
    18 : 'bert-select-mod-gatv1-edge',
    19 : 'bert-select-mod-gatv2-edge',
    20 : 'bert-select-mod-rgat',
    21 : 'bert-select-mod-egat',
}
selected_combination = [0]
# selected_combination = [3]

<h4> Getting BERT and GAT outputs for all sets

In [8]:
# trainFeaturesList[0][0].shape

In [9]:
def concatenate_tensors(tensor_list):
    return torch.cat(tensor_list, dim=0)

def import_h_prime(file_paths):
    featuresList = []
    attList = []
    for file_path in file_paths:
        with open(file_path, 'rb') as file:
            data = pickle.load(file)
#             print("Check... ", len(data))
            if isinstance(data, list):
                print(type(data), " instance of list: ", len(data))
                featuresList.append(concatenate_tensors(data))
            else:
                print(type(data), " instance of tensor, ", data.shape)
                featuresList.append(data)
                
    return featuresList
    
trainFeaturesList = import_h_prime(train_file_paths)

testFeaturesList = import_h_prime(test_file_paths)

valFeaturesList = import_h_prime(val_file_paths)


<class 'list'>  instance of list:  2160
<class 'list'>  instance of list:  2160
<class 'list'>  instance of list:  2160
<class 'list'>  instance of list:  2160
<class 'list'>  instance of list:  2160
<class 'list'>  instance of list:  2160
<class 'list'>  instance of list:  2160
<class 'list'>  instance of list:  577
<class 'list'>  instance of list:  577
<class 'list'>  instance of list:  577
<class 'list'>  instance of list:  577
<class 'list'>  instance of list:  577
<class 'list'>  instance of list:  577
<class 'list'>  instance of list:  577
<class 'list'>  instance of list:  270
<class 'list'>  instance of list:  270
<class 'list'>  instance of list:  270
<class 'list'>  instance of list:  270
<class 'list'>  instance of list:  270
<class 'list'>  instance of list:  270
<class 'list'>  instance of list:  270


In [10]:
trainFeaturesList[0].shape

torch.Size([12840, 768])

In [11]:
def getNodalAttn():
    nodalAttList = []
    nodalAttList.append(False)
    nodalAttList.append(False)
    nodalAttList.append(False)
    nodalAttList.append(False)
    nodalAttList.append(True)
    nodalAttList.append(True)
    nodalAttList.append(True)
    nodalAttList.append(True)
    nodalAttList.append(True)
    nodalAttList.append(True)
    nodalAttList.append(True)
    nodalAttList.append(True)
    nodalAttList.append(True)
    nodalAttList.append(True)
    nodalAttList.append(True)
    nodalAttList.append(True)
    nodalAttList.append(True)
    nodalAttList.append(True)
    nodalAttList.append(True)
    nodalAttList.append(True)
    nodalAttList.append(True)
    nodalAttList.append(True)
    return nodalAttList

In [12]:
nodalAttn = getNodalAttn()

EDA

In [13]:
# # Checking the structure of graph
# for n in range(10):
#     tensor_data_np = tensor_utterances[n].detach().numpy()

#     # Plot the data
#     plt.figure(figsize=(10, 5))
#     plt.plot(range(len(tensor_data_np)), tensor_data_np)
#     plt.title('Line Graph of Tensor Data')
#     plt.xlabel('Index')
#     plt.ylabel('Value')
#     plt.show()


In [14]:
# # Normalize the h' (1st GAT)
# data = cherry_picked_nodes.detach().numpy()
# data_normalized = data / np.linalg.norm(data, axis=1, keepdims=True)

# # Compute pairwise cosine similarities
# similarities = cosine_similarity(data_normalized)

# # Print or analyze the similarity matrix
# # print(similarities)
# plt.hist(similarities.flatten(), bins=50, density=True)
# plt.title('Distribution of Cosine Similarities')
# plt.xlabel('Cosine Similarity')
# plt.ylabel('Frequency')
# plt.show()


In [15]:
# # Normalize the h' (2nd GAT)
# data = all_node_feats.detach().numpy()
# data_normalized = data / np.linalg.norm(data, axis=1, keepdims=True)

# # Compute pairwise cosine similarities
# similarities = cosine_similarity(data_normalized)

# # Print or analyze the similarity matrix
# # print(similarities)
# plt.hist(similarities.flatten(), bins=50, density=True)
# plt.title('Distribution of Cosine Similarities')
# plt.xlabel('Cosine Similarity')
# plt.ylabel('Frequency')
# plt.show()


In [16]:
# # Normalize the u' or updated_representations
# data = tensor_utterances.detach().numpy()
# data_normalized = data / np.linalg.norm(data, axis=1, keepdims=True)

# # Compute pairwise cosine similarities
# similarities = cosine_similarity(data_normalized)

# plt.hist(similarities.flatten(), bins=50, density=True)
# plt.title('Distribution of Cosine Similarities')
# plt.xlabel('Cosine Similarity')
# plt.ylabel('Frequency')
# plt.show()


<h3> Feature Selection and creating data combination for classifiers

Define select feature function

In [17]:
def get_norm_features(encoded_features):
    scaler = MinMaxScaler()
    features_scaled = scaler.fit_transform(encoded_features)
    return torch.tensor(features_scaled)

def get_selected_features(encoded_features, labels, top_n):
    if torch.is_tensor(encoded_features):
        encoded_features = encoded_features.detach().cpu().numpy()
    
    scaler = MinMaxScaler()
    features_scaled = scaler.fit_transform(encoded_features)

    top_features_by_class = {}
    top_scores = {}

    for label in range(7):
        # Create a binary mask indicating instances belonging to the current class
        mask = (labels == label)

        selector = SelectKBest(score_func=f_classif, k=top_n) 
        selector.fit(features_scaled, mask)  

        top_features_indices = np.argsort(selector.scores_)[-top_n:]
        scores = selector.scores_[top_features_indices]

        top_features_by_class[label] = top_features_indices
        top_scores[label] = scores

    concatenated_features_set = set()
    for label, indices in top_features_by_class.items():
        concatenated_features_set.update(indices)

    concatenated_features_indices = list(concatenated_features_set)

    concatenated_features_indices = np.array(concatenated_features_indices)


    selected_features = encoded_features[:, concatenated_features_indices]

    return selected_features, concatenated_features_indices

In [18]:
class Autoencoder(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(hidden_dim, input_dim),
            nn.ReLU()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

# Function to train the autoencoder
def train_autoencoder(encoded_features, hidden_dim=100, num_epochs=20, lr=0.001):
    input_dim = encoded_features.shape[1]
    autoencoder = Autoencoder(input_dim, hidden_dim)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(autoencoder.parameters(), lr=lr)
    
    X_tensor = torch.tensor(encoded_features, dtype=torch.float32)
    train_loader = torch.utils.data.DataLoader(X_tensor, batch_size=32, shuffle=True)
    
    for epoch in tqdm(range(num_epochs), desc="Training Autoencoder", unit="epoch"):
        total_loss = 0.0
        for data in train_loader:
            inputs = data
            optimizer.zero_grad()
            _, decoded = autoencoder(inputs)
            loss = criterion(decoded, inputs)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
    
    with torch.no_grad():
        encoded_features, _ = autoencoder(torch.tensor(encoded_features, dtype=torch.float32))
    
    return encoded_features, autoencoder

def get_selected_features_autoencoder(autoencoder, encoded_features, labels, top_n=100):
    if torch.is_tensor(encoded_features):
        encoded_features = encoded_features.detach().cpu().numpy()

    reduced_features = autoencoder.encoder(torch.tensor(encoded_features, dtype=torch.float32)).detach().numpy()
    
    scaler = MinMaxScaler()
    features_scaled = scaler.fit_transform(reduced_features)
    
    top_features_by_class = {}
    top_scores = {}

    for label in range(7):
        mask = (labels == label)
        selector = chi2(features_scaled, mask)
        top_features_indices = np.argsort(selector[0])[-top_n:]
        scores = selector[0][top_features_indices]

        top_features_by_class[label] = top_features_indices
        top_scores[label] = scores

    concatenated_features_set = set()
    for label, indices in top_features_by_class.items():
        concatenated_features_set.update(indices)

    concatenated_features_indices = list(concatenated_features_set)
    concatenated_features_indices = np.array(concatenated_features_indices)

    selected_features = reduced_features[:, concatenated_features_indices]

    return selected_features, concatenated_features_indices

In [19]:
# pca = PCA(n_components=2)
# pca_result = pca.fit_transform(selected_features.detach().numpy())

# # Plot the PCA result with color-coded labels
# plt.figure(figsize=(8, 6))
# for label in np.unique(Y_train):
#     indices = Y_train == label
#     plt.scatter(pca_result[indices, 0], pca_result[indices, 1], label=f'{label_decoder[label]}', alpha=0.5)
#     plt.title('PCA Visualization of Selected Utterance Embeddings (Train) with Color-Coded Labels')
#     plt.xlabel('Principal Component 1')
#     plt.ylabel('Principal Component 2')
#     plt.legend()
#     plt.grid(True)
#     plt.show()

3d plottly

In [20]:
# X_train = selected_features
# X_train = X_train / np.linalg.norm(X_train, axis=1, keepdims=True)
# # Perform T-SNE dimensionality reduction
# tsne = TSNE(n_components=3, random_state=42)
# X_tsne = tsne.fit_transform(X_train)

# # Create a Plotly scatter plot
# fig = go.Figure(data=[go.Scatter3d(
#     x=X_tsne[:, 0],
#     y=X_tsne[:, 1],
#     z=X_tsne[:, 2],
#     mode='markers',
#     marker=dict(
#         size=3,
#         color=Y_train,  # Assuming Y_train contains labels for coloring
#         colorscale='Viridis',  # You can choose a different colorscale
#         opacity=0.8
#     )
# )])

# # Update layout
# fig.update_layout(title='3D T-SNE Plot', autosize=False,
#                   width=800, height=800)

# # Show the plot
# fig.show()

In [21]:
# Save the plot as an HTML file
# pio.write_html(fig, '3d_tsne_plot.html')

Now prepare the data that will be ued to train the classifier, there are 20 combinations. And pick top 7 combinations yielding top F1 weighted-score

In [22]:
# for item in trainFeaturesList[6][:10]:
#     print(item)

In [23]:
y_train.shape

torch.Size([12840])

In [24]:
trainList = []
testList = []
valList = []

file_path1 = "data/dump/" + dataset_path + "/BERT_data_for_classifier/trainList.pkl"
file_path2 = "data/dump/" + dataset_path + "/BERT_data_for_classifier/testList.pkl"
file_path3 = "data/dump/" + dataset_path + "/BERT_data_for_classifier/valList.pkl"

checkFile1 = os.path.isfile(file_path1)
checkFile2 = os.path.isfile(file_path2)
checkFile3 = os.path.isfile(file_path3)

if checkFile1 and checkFile2 and checkFile3: 
    with open(file_path1, "rb") as file:
        trainList = pickle.load(file)
    with open(file_path2, "rb") as file:
        testList = pickle.load(file)
    with open(file_path3, "rb") as file:
        valList = pickle.load(file)
else:
#     trainFeaturesList.append(data)
    #1
    trainList.append(trainFeaturesList[0])
    testList.append(testFeaturesList[0])
    valList.append(valFeaturesList[0])
    #2
    selectedTrainFeatures1a, indicesFeatures1a = get_selected_features(trainFeaturesList[0], y_train, 16)
    selectedTestFeatures1a = testFeaturesList[0][:, indicesFeatures1a]
    selectedValFeatures1a = valFeaturesList[0][:, indicesFeatures1a]
    trainList.append(selectedTrainFeatures1a)
    testList.append(selectedTestFeatures1a)
    valList.append(selectedValFeatures1a)
    #3
    selectedTrainFeatures1b, indicesFeatures1b = get_selected_features(trainFeaturesList[0], y_train, 32)
    selectedTestFeatures1b = testFeaturesList[0][:, indicesFeatures1b]
    selectedValFeatures1b = valFeaturesList[0][:, indicesFeatures1b]
    trainList.append(selectedTrainFeatures1b)
    testList.append(selectedTestFeatures1b)
    valList.append(selectedValFeatures1b)
    #4
    selectedTrainFeatures1a, indicesFeatures1a = get_selected_features(trainFeaturesList[0], y_train, 64)
    selectedTestFeatures1a = testFeaturesList[0][:, indicesFeatures1a]
    selectedValFeatures1a = valFeaturesList[0][:, indicesFeatures1a]
    trainList.append(selectedTrainFeatures1a)
    testList.append(selectedTestFeatures1a)
    valList.append(selectedValFeatures1a)
    #5
    trainList.append(trainFeaturesList[1])
    testList.append(testFeaturesList[1])
    valList.append(valFeaturesList[1])
    #6
    selectedTrainFeatures2, indicesFeatures2 = get_selected_features(trainFeaturesList[1], y_train, 12)
    selectedTestFeatures2 = testFeaturesList[1][:, indicesFeatures2]
    selectedValFeatures2 = valFeaturesList[1][:, indicesFeatures2]
    trainList.append(selectedTrainFeatures2)
    testList.append(selectedTestFeatures2)
    valList.append(selectedValFeatures2)
    #7
    trainList.append(trainFeaturesList[2])
    testList.append(testFeaturesList[2])
    valList.append(valFeaturesList[2])
    #8
    selectedTrainFeatures3, indicesFeatures3 = get_selected_features(trainFeaturesList[2], y_train, 12)
    selectedTestFeatures3 = testFeaturesList[2][:, indicesFeatures3]
    selectedValFeatures3 = valFeaturesList[2][:, indicesFeatures3]
    trainList.append(selectedTrainFeatures3)
    testList.append(selectedTestFeatures3)
    valList.append(selectedValFeatures3)
    #9
    trainList.append(trainFeaturesList[3])
    testList.append(testFeaturesList[3])
    valList.append(valFeaturesList[3])
    #10
    selectedTrainFeatures4, indicesFeatures4 = get_selected_features(trainFeaturesList[3], y_train, 12)
    selectedTestFeatures4 = testFeaturesList[3][:, indicesFeatures4]
    selectedValFeatures4 = valFeaturesList[3][:, indicesFeatures4]
    trainList.append(selectedTrainFeatures4)
    testList.append(selectedTestFeatures4)
    valList.append(selectedValFeatures4)
    #11
    trainList.append(trainFeaturesList[4])
    testList.append(testFeaturesList[4])
    valList.append(valFeaturesList[4])
    #12
    selectedTrainFeatures5, indicesFeatures5 = get_selected_features(trainFeaturesList[4], y_train, 12)
    selectedTestFeatures5 = testFeaturesList[4][:, indicesFeatures5]
    selectedValFeatures5 = valFeaturesList[4][:, indicesFeatures5]
    trainList.append(selectedTrainFeatures5)
    testList.append(selectedTestFeatures5)
    valList.append(selectedValFeatures5)
    #13
    trainList.append(trainFeaturesList[5])
    testList.append(testFeaturesList[5])
    valList.append(valFeaturesList[5])
    #14
    selectedTrainFeatures6, indicesFeatures6 = get_selected_features(trainFeaturesList[5], y_train, 12)
    selectedTestFeatures6 = testFeaturesList[5][:, indicesFeatures6]
    selectedValFeatures6 = valFeaturesList[5][:, indicesFeatures6]
    trainList.append(selectedTrainFeatures6)
    testList.append(selectedTestFeatures6)
    valList.append(selectedValFeatures6)
    #15
    trainList.append(trainFeaturesList[6])
    testList.append(testFeaturesList[6])
    valList.append(valFeaturesList[6])
    #16
    selectedTrainFeatures7, indicesFeatures7 = get_selected_features(trainFeaturesList[6], y_train, 12)
    selectedTestFeatures7 = testFeaturesList[6][:, indicesFeatures7]
    selectedValFeatures7 = testFeaturesList[6][:, indicesFeatures7]
    trainList.append(selectedTrainFeatures7)
    testList.append(selectedTestFeatures7)
    valList.append(selectedValFeatures7)
    selectedNormTrainFeatures1 = get_norm_features(selectedTrainFeatures1b)
    selectedNormTestFeatures1 = get_norm_features(selectedTestFeatures1b)
    selectedNormValFeatures1 = get_norm_features(selectedValFeatures1b)

    #17
    trainNormFeatures2 = get_norm_features(trainFeaturesList[1].detach().numpy())
    testNormFeatures2 = get_norm_features(testFeaturesList[1].detach().numpy())
    valNormFeatures2 = get_norm_features(valFeaturesList[1].detach().numpy())
    concatenatedTrainFeatures2 = torch.cat((selectedNormTrainFeatures1, trainNormFeatures2), dim=1)
    concatenatedTestFeatures2 = torch.cat((selectedNormTestFeatures1, testNormFeatures2), dim=1)
    concatenatedValFeatures2 = torch.cat((selectedNormValFeatures1, valNormFeatures2), dim=1)
    trainList.append(concatenatedTrainFeatures2)
    testList.append(concatenatedTestFeatures2)
    valList.append(concatenatedValFeatures2)
    #18
    trainNormFeatures3 = get_norm_features(trainFeaturesList[2].detach().numpy())
    testNormFeatures3 = get_norm_features(testFeaturesList[2].detach().numpy())
    valNormFeatures3 = get_norm_features(valFeaturesList[2].detach().numpy())
    concatenatedTrainFeatures3 = torch.cat((selectedNormTrainFeatures1, trainNormFeatures3), dim=1)
    concatenatedTestFeatures3 = torch.cat((selectedNormTestFeatures1, testNormFeatures3), dim=1)
    concatenatedValFeatures3 = torch.cat((selectedNormValFeatures1, valNormFeatures3), dim=1)
    trainList.append(concatenatedTrainFeatures3)
    testList.append(concatenatedTestFeatures3)
    valList.append(concatenatedValFeatures3)
    #19
    trainNormFeatures4 = get_norm_features(trainFeaturesList[3].detach().numpy())
    testNormFeatures4 = get_norm_features(testFeaturesList[3].detach().numpy())
    valNormFeatures4 = get_norm_features(valFeaturesList[3].detach().numpy())
    concatenatedTrainFeatures4 = torch.cat((selectedNormTrainFeatures1, trainNormFeatures4), dim=1)
    concatenatedTestFeatures4 = torch.cat((selectedNormTestFeatures1, testNormFeatures4), dim=1)
    concatenatedValFeatures4 = torch.cat((selectedNormValFeatures1, valNormFeatures4), dim=1)
    trainList.append(concatenatedTrainFeatures4)
    testList.append(concatenatedTestFeatures4)
    valList.append(concatenatedValFeatures4)
    #20
    trainNormFeatures5 = get_norm_features(trainFeaturesList[4].detach().numpy())
    testNormFeatures5 = get_norm_features(testFeaturesList[4].detach().numpy())
    valNormFeatures5 = get_norm_features(valFeaturesList[4].detach().numpy())
    concatenatedTrainFeatures5 = torch.cat((selectedNormTrainFeatures1, trainNormFeatures5), dim=1)
    concatenatedTestFeatures5 = torch.cat((selectedNormTestFeatures1, testNormFeatures5), dim=1)
    concatenatedValFeatures5 = torch.cat((selectedNormValFeatures1, valNormFeatures5), dim=1)
    trainList.append(concatenatedTrainFeatures5)
    testList.append(concatenatedTestFeatures5)
    valList.append(concatenatedValFeatures5)

    #21
    trainNormFeatures6 = get_norm_features(trainFeaturesList[5].detach().numpy())
    testNormFeatures6 = get_norm_features(testFeaturesList[5].detach().numpy())
    valNormFeatures6 = get_norm_features(valFeaturesList[5].detach().numpy())
    concatenatedTrainFeatures6 = torch.cat((selectedNormTrainFeatures1, trainNormFeatures6), dim=1)
    concatenatedTestFeatures6 = torch.cat((selectedNormTestFeatures1, testNormFeatures6), dim=1)
    concatenatedValFeatures6 = torch.cat((selectedNormValFeatures1, valNormFeatures6), dim=1)
    trainList.append(concatenatedTrainFeatures6)
    testList.append(concatenatedTestFeatures6)
    valList.append(concatenatedValFeatures6)

    #22
    trainNormFeatures7 = get_norm_features(trainFeaturesList[6].detach().numpy())
    testNormFeatures7 = get_norm_features(testFeaturesList[6].detach().numpy())
    valNormFeatures7 = get_norm_features(valFeaturesList[6].detach().numpy())
    concatenatedTrainFeatures7 = torch.cat((selectedNormTrainFeatures1, trainNormFeatures7), dim=1)
    concatenatedTestFeatures7 = torch.cat((selectedNormTestFeatures1, testNormFeatures7), dim=1)
    concatenatedValFeatures7 = torch.cat((selectedNormValFeatures1, valNormFeatures7), dim=1)
    trainList.append(concatenatedTrainFeatures7)
    testList.append(concatenatedTestFeatures7)
    valList.append(concatenatedValFeatures7)

    trainList_tensors = [torch.tensor(item) for item in trainList]
    testList_tensors = [torch.tensor(item) for item in testList]
    valList_tensors = [torch.tensor(item) for item in valList]

    with open(file_path1, 'wb') as file:
        pickle.dump(trainList_tensors, file)
    with open(file_path2, 'wb') as file:
        pickle.dump(testList_tensors, file)
    with open(file_path3, 'wb') as file:
        pickle.dump(valList_tensors, file)

In [25]:
trainList[1].shape

torch.Size([12840, 96])

1. Prep data - normalize and create data loader

In [26]:
# for item in trainFeaturesList:
#     print(item.shape)

In [27]:
def to_tensor(data):
    if isinstance(data, torch.Tensor):
        return data
    elif isinstance(data, np.ndarray):
        return torch.tensor(data)
    else:
        raise TypeError(f"Unsupported data type: {type(data)}")
        
def prep_data(features, labels, isOversample):
    num_classes = 7

    if isOversample:
        X_set, Y_set = oversample_data(features, labels, num_classes)
    else:
        X_set, Y_set = features, labels

    if isinstance(X_set, torch.Tensor):
        X_tensor = X_set.float()
    else:
        X_tensor = torch.tensor(X_set, dtype=torch.float32)
    
    if isinstance(Y_set, torch.Tensor):
        Y_tensor = Y_set.long()
    else:
        Y_tensor = torch.tensor(Y_set, dtype=torch.long)

    unique_labels, label_counts = np.unique(Y_set, return_counts=True)
    dataset = TensorDataset(X_tensor, Y_tensor)

    return X_tensor, Y_tensor

2. Training

Backup

In [28]:
# def model_train1(X_set, Y_set, num_epochs=20, batch_size=32, loss_difference_threshold=0.01, 
#                  hidden_dims=[256, 128], dropout_rate=0.5, lr=0.0001, 
#                  optimizer_class=optim.Adam, criterion_class=nn.CrossEntropyLoss, matchAtt
#                  ):
#     output_dim = 7  # Number of classes
#     model = MyNetwork(len(X_set[0]), hidden_dims, output_dim, dropout_rate)
#     criterion = criterion_class()
#     optimizer = optimizer_class(model.parameters(), lr=lr)
#     loss_history = []
#     accuracy_history = []
# #     print_interval = 1  # Print tqdm every epoch
#     previous_loss = float('inf')

#     # Create dataset and dataloader
#     dataset = TensorDataset(X_set, Y_set)
#     dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
#     epoch_num = num_epochs
#     for epoch in range(num_epochs):
#         total_loss = 0.0
#         correct_predictions = 0
#         total_instances = 0
#         with tqdm(total=len(dataloader), desc=f'Epoch {epoch+1}/{num_epochs}', leave=False) as pbar:
#             for inputs, labels in dataloader:
#                 inputs = inputs.float()  # Ensure inputs are float32
#                 labels = labels.long()   # Ensure labels are long
#                 optimizer.zero_grad()
# #                 TODO
#                 outputs = model(inputs)
#                 outputs = outputs.squeeze()
#                 labels = labels.squeeze()
#                 loss = criterion(outputs, labels)
                
#                 # Check for NaN loss values
#                 if torch.isnan(loss):
#                     print("NaN loss encountered. Skipping this batch.")
#                     break
                
#                 # Apply gradient clipping
#                 torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
                
#                 loss.backward()
#                 optimizer.step()
#                 total_loss += loss.item()
#                 _, predicted = torch.max(outputs, dim=1)
#                 correct_predictions += (predicted == labels).sum().item()
#                 total_instances += labels.size(0)
#                 pbar.update(1)

#         epoch_loss = total_loss / total_instances
#         epoch_accuracy = correct_predictions / total_instances
#         loss_history.append(epoch_loss)
#         accuracy_history.append(epoch_accuracy)

#         if epoch > 0 and abs(epoch_loss - previous_loss) < loss_difference_threshold:
#             epoch_num = epoch
#             break

#         previous_loss = epoch_loss

#     return model, epoch_num


In [29]:
def shuffle_data(X_set, Y_set):
    indices = np.arange(len(X_set))
    np.random.shuffle(indices)
    return X_set[indices], Y_set[indices]

In [30]:
def model_train1(X_set=None, Y_set=None, num_epochs=50, loss_difference_threshold=0.0001, 
                 hidden_dims=[64, 32], dropout_rate=0.5, lr=0.0001, 
                 optimizer_class=optim.Adam, criterion_class=nn.CrossEntropyLoss, 
                 nodalAtt=None, umask=None, seq_len=None, no_cuda=True, ranges=None):
    
    input_dim = X_set.shape[1] if len(X_set) > 0 else 0
    model = MyNetwork(input_dim=input_dim, hidden_dims=hidden_dims, output_dim=7, dropout_rate=0.5)
    criterion = criterion_class()
    optimizer = optimizer_class(model.parameters(), lr=lr)
    loss_history = []
    accuracy_history = []
    previous_loss = float('inf')

    epoch_num = num_epochs

    for epoch in range(num_epochs):
        total_loss = 0.0
        correct_predictions = 0
        total_instances = 0
        
        # Shuffle the dataset at the beginning of each epoch
        X_set, Y_set = shuffle_data(X_set, Y_set)
        
        with tqdm(total=len(ranges), desc=f'Epoch {epoch+1}/{num_epochs}', leave=False) as pbar:
            for i, (start, end) in enumerate(ranges):
                inputs = X_set[start:end+1].float()
                labels = Y_set[start:end+1].long()
                
                optimizer.zero_grad()
                last_idx = seq_len[i][0]
                
                umask_slice = umask[i][0][:last_idx].unsqueeze(0)
#                 print("inputs: ", inputs)
#                 print("seq_len: ", seq_len[i])
#                 print("umask_slice: ", umask_slice)
                outputs = model(inputs, nodalAtt, seq_len[i], umask_slice)
                outputs = outputs.squeeze()
                labels = labels.squeeze()
                
                log_prob = F.log_softmax(outputs, 1)
                
                loss = criterion(log_prob, labels)
                
                # Check for NaN loss values
                if torch.isnan(loss):
                    print("NaN loss encountered. Skipping this batch.")
                    break
                
                # Apply gradient clipping
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
                
                if nodalAtt:
                    loss.backward(retain_graph=True)
                else:
                    loss.backward(retain_graph=False)
                
                optimizer.step()
                total_loss += loss.item()
                _, predicted = torch.max(outputs, dim=1)
                correct_predictions += (predicted == labels).sum().item()
                total_instances += labels.size(0)
                pbar.update(1)

        epoch_loss = total_loss / total_instances
        epoch_accuracy = correct_predictions / total_instances
        loss_history.append(epoch_loss)
        accuracy_history.append(epoch_accuracy)

        if epoch > 0 and abs(epoch_loss - previous_loss) < loss_difference_threshold:
            epoch_num = epoch
            break

        previous_loss = epoch_loss

    return model, epoch_num

Backup

In [31]:
def model_train2(X_set=None, Y_set=None, num_epochs=20, loss_difference_threshold=0.001, 
                 hidden_dims=128, dropout_rate=0.5, lr=0.0001, 
                 optimizer_class=optim.Adam, criterion_class=nn.CrossEntropyLoss, 
                 nodalAtt=None, umask=None, seq_len=None, no_cuda=True, ranges=None):
    
    output_dim = 7  # Number of classes
    input_dim = X_set.shape[1] if len(X_set) > 0 else 0
    model = FCClassifier(input_dim, hidden_dims, output_dim, dropout_rate)
    criterion = criterion_class()
    optimizer = optimizer_class(model.parameters(), lr=lr)
    loss_history = []
    accuracy_history = []
    previous_loss = float('inf')

    epoch_num = num_epochs

    for epoch in range(num_epochs):
        total_loss = 0.0
        correct_predictions = 0
        total_instances = 0
        
        # Shuffle the dataset at the beginning of each epoch
        X_set, Y_set = shuffle_data(X_set, Y_set)
        
        with tqdm(total=len(ranges), desc=f'Epoch {epoch+1}/{num_epochs}', leave=False) as pbar:
            for i, (start, end) in enumerate(ranges):
                inputs = X_set[start:end+1].float()
                labels = Y_set[start:end+1].long()
                
                optimizer.zero_grad()
                last_idx = seq_len[i][0]
                
                umask_slice = umask[i][0][:last_idx].unsqueeze(0)

#                 print("inputs: ", inputs)
#                 print("seq_len: ", seq_len[i])
#                 print("umask_slice: ", umask_slice)
                outputs = model(inputs, nodalAtt, seq_len[i], umask_slice)
                outputs = outputs.squeeze()
                labels = labels.squeeze()
#                 print(outputs.shape, labels.shape)
                log_prob = F.log_softmax(outputs, 1)
                
                loss = criterion(log_prob, labels)
                
                # Check for NaN loss values
                if torch.isnan(loss):
                    print("NaN loss encountered. Skipping this batch.")
                    break
                
                # Apply gradient clipping
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
                if nodalAtt:
                    loss.backward(retain_graph=True)
                else:
                    loss.backward(retain_graph=False)
                    
                optimizer.step()
                
                total_loss += loss.item()
                _, predicted = torch.max(log_prob, dim=1)
                correct_predictions += (predicted == labels).sum().item()
                total_instances += labels.size(0)
                pbar.update(1)
                
        epoch_loss = total_loss / total_instances
        epoch_accuracy = correct_predictions / total_instances
        loss_history.append(epoch_loss)
        accuracy_history.append(epoch_accuracy)
        
        if epoch > 0:
            loss_diff = abs(epoch_loss - previous_loss)
            if loss_diff < loss_difference_threshold:
                print(f"Training stopped early at epoch {epoch+1}.")
                print(f"Loss difference ({loss_diff}) is below the threshold ({loss_difference_threshold}).")
                epoch_num = epoch + 1
                break
        
        previous_loss = epoch_loss
    
    return model, epoch_num

In [32]:
def classify_emotions(model=None, X_set=None, Y_set=None, typeSet=None, 
                      isSimpleFC=False, i_dict=None,
                      nodalAtt=None, umask=None, seq_len=None, no_cuda=True, ranges=None):
    # Ensure the model is in evaluation mode
    model.eval()
    
    # Initialize empty lists to store predictions and true labels
    all_predictions = []
    all_labels = []
    i = 0
    # Iterate over the given ranges
    for start, end in ranges:
        X_batch = X_set[start:end+1].float()
        Y_batch = Y_set[start:end+1].long()

#         if X_batch.dtype != torch.float32:
#             X_batch = X_batch.float()

        # Use no_grad to save memory and computations
        with torch.no_grad():
            inputs = X_batch.to('cuda' if torch.cuda.is_available() and not no_cuda else 'cpu')
            labels = Y_batch.to('cuda' if torch.cuda.is_available() and not no_cuda else 'cpu')
            last_idx = seq_len[i][0]
            
            umask_slice = umask[i][0][:last_idx].unsqueeze(0)

            outputs = model(inputs, nodalAtt, seq_len[i], umask_slice)

            outputs = outputs.squeeze()
            labels = labels.squeeze()

            log_prob = F.log_softmax(outputs, 1)

            _, predicted = torch.max(log_prob, 1)

            # Append the predictions and labels to the lists
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            i = i+1

    df_predictions = pd.DataFrame({
        'true_label': all_labels,
        'predicted_label': all_predictions
    })
    
    
    file_name = f"data/dump/{dataset_path}/BERT_data_for_classifier/{dictKey[i_dict]}_predictedTest.pkl"
    with open(file_name, 'wb') as file:
        pickle.dump(df_predictions, file)
        
    # Generate the classification report
    report = classification_report(all_labels, all_predictions, target_names=label_decoder.values(), 
                                   output_dict=True, zero_division=0, digits=4)
    print(report)
    # Calculate the required metrics
    accuracy = report['accuracy']
    recall = report['weighted avg']['recall']
    weighted_f1 = report['weighted avg']['f1-score']
    f1_micro = report.get('micro avg', {}).get('f1-score', accuracy)
    f1_macro = report.get('macro avg', {}).get('f1-score', 0.0) 

    if typeSet == "validation":
        print("Classified: ", dictKey[i_dict])
    
    return dictKey[i_dict], typeSet, isSimpleFC, accuracy, recall, weighted_f1, f1_micro, f1_macro

In [33]:
# i = 0
# for trainSet, testSet, valSet, _, _, _ in tqdm(dataLoader, desc="Encoding Progress", unit="batch"):
#     print(i, type(trainSet))
#     if isinstance(trainSet, list):
#         print(type(trainSet[0]))
#         sample = trainSet[0]
#         print(sample.shape)
#     else:
#         print(trainSet.squeeze(0).shape)
#     i = i+1

In [34]:
def getGraphComponents(file_path):
    checkFile = os.path.isfile(file_path)
    if checkFile:
        with open(file_path, 'rb') as file:
            umask, \
            seq_lengths,\
            features, \
            edge_index, \
            edge_norm, \
            edge_type, \
            edge_index_lengths = pickle.load(file)  
    else:
        return None
    
    return umask, \
            seq_lengths,\
            features, \
            edge_index, \
            edge_norm, \
            edge_type, \
            edge_index_lengths

In [35]:
file_path1 = 'embed/' + dataset_path + '/pre_h_prime_BERT_train.pkl'
file_path2 = 'embed/' + dataset_path + '/pre_h_prime_BERT_test.pkl'
file_path3 = 'embed/' + dataset_path + '/pre_h_prime_BERT_dev.pkl'

train_umask, train_seq_lengths, train_features, \
    _, _, _, _ = getGraphComponents(file_path1)

test_umask, test_seq_lengths, test_features, \
    _, _, _, _ = getGraphComponents(file_path2)

val_umask, val_seq_lengths, val_features, \
    _, _, _, _ = getGraphComponents(file_path3)

UpdateJune 16

In [36]:
print(len(train_umask), len(train_seq_lengths))
print(train_seq_lengths[1])

2160 2160
[7]


In [37]:
def getSpeakersAndRanges(file_path):
    with open(file_path, "rb") as file:
        encodedSpeakers, ranges = pickle.load(file)
    file.close()
    return encodedSpeakers, ranges

file_path1 = "data/dump/" + dataset_path + "/speaker_encoder_train.pkl"
file_path2 = "data/dump/" + dataset_path + "/speaker_encoder_test.pkl"
file_path3 = "data/dump/" + dataset_path + "/speaker_encoder_dev.pkl"

encodedSpeakersTrain, rangesTrain = getSpeakersAndRanges(file_path1)
encodedSpeakersTest, rangesTest = getSpeakersAndRanges(file_path2)
encodedSpeakersDev, rangesDev = getSpeakersAndRanges(file_path3)

In [38]:
# for item in rangesTrain:
#     if item[0]+1 == item[1]:
#         print("Dup detected")

Sample run

Verifying the attention in batch before training

In [39]:
# class MatchingAttention(nn.Module):
#     def __init__(self, mem_dim, cand_dim, alpha_dim, att_type='general2'):
#         super(MatchingAttention, self).__init__()
#         self.mem_dim = mem_dim
#         self.cand_dim = cand_dim
#         self.alpha_dim = alpha_dim
#         self.att_type = att_type

#         if self.att_type == 'general2':
#             self.transform = nn.Linear(self.mem_dim, self.cand_dim * self.alpha_dim)

#     def forward(self, M, x, mask):
#         M_ = M.permute(1, 2, 0)  # (batch, mem_dim, seq_len)
#         x_ = self.transform(x).unsqueeze(1)  # (batch, 1, cand_dim * alpha_dim)
#         mask_ = mask.unsqueeze(2).repeat(1, 1, self.mem_dim).transpose(1, 2)  # (batch, mem_dim, seq_len)
        
#         M_ = M_ * mask_
#         alpha = torch.bmm(x_, M_)  # (batch, 1, seq_len)
        
#         alpha = F.softmax(alpha, dim=-1)  # Apply softmax to get attention weights
#         attended = torch.bmm(alpha, M.permute(1, 0, 2))  # (batch, 1, mem_dim)
        
#         return attended.squeeze(1), alpha
    
# def attentive_node_features(emotions, seq_lengths, umask, matchatt_layer):
#     max_len = max(seq_lengths)
#     batch_size = len(seq_lengths)
#     mem_dim = emotions.size(1)

#     padded_emotions = []
#     for i in range(batch_size):
#         length = seq_lengths[i]
#         # Assuming emotions is already a 2D tensor of shape (seq_len, mem_dim)
#         padded_emotion = F.pad(emotions[:length], (0, 0, 0, max_len - length), "constant", 0)
#         padded_emotions.append(padded_emotion)

#     emotions_padded = torch.stack(padded_emotions, dim=1)  # (max_len, batch_size, mem_dim)
    
#     att_emotions = []
#     alpha_list = []
#     for t in range(max_len):
#         att_em, alpha = matchatt_layer(emotions_padded, emotions_padded[t], umask)
#         att_emotions.append(att_em)
#         alpha_list.append(alpha)

#     att_emotions = torch.stack(att_emotions, dim=0)  # (max_len, batch_size, mem_dim)

#     # Remove the singleton dimension for batch size 1
#     att_emotions = att_emotions.squeeze(1)  # (seq_len, mem_dim)

#     return att_emotions, alpha_list
    
# class MyNetwork(nn.Module):
#     def __init__(self, input_dim, hidden_dims, output_dim, dropout_rate):
#         super(MyNetwork, self).__init__()
#         self.fc1 = nn.Linear(input_dim, hidden_dims[0])  # Adjust input_dim to match flattened shape
#         self.activation1 = nn.ReLU()
#         self.fc2 = nn.Linear(hidden_dims[0], hidden_dims[1])
#         self.activation2 = nn.ReLU()
#         self.dropout = nn.Dropout(dropout_rate)
#         self.fc3 = nn.Linear(hidden_dims[1], output_dim)
#         self.matchatt = MatchingAttention(mem_dim=64, cand_dim=64, alpha_dim=1, att_type='general2')

#     def forward(self, x, nodalAtt, seq_lengths, umask):
#         if nodalAtt:
#             att_emotions, _ = attentive_node_features(x, seq_lengths, umask, self.matchatt)

#             # Reshape att_emotions to have a 2D shape (batch_size, input_dim)
#             att_emotions = att_emotions.view(att_emotions.size(0), -1)
        
#         x = self.fc1(att_emotions)
#         x = self.activation1(x)
#         x = self.fc2(x)
#         x = self.activation2(x)
#         x = self.dropout(x)
#         x = self.fc3(x)
        
#         return x

# class FCClassifier(torch.nn.Module):
#     def __init__(self, input_dim, hidden_dim, output_dim, dropout_rate):
#         super(FCClassifier, self).__init__()
#         self.linear1 = nn.Linear(input_dim, hidden_dim)
#         self.dropout = nn.Dropout(dropout_rate)
#         self.linear2 = nn.Linear(hidden_dim, output_dim)
#         self.matchatt = MatchingAttention(mem_dim=input_dim, cand_dim=input_dim, alpha_dim=1, att_type='general2')
#     def forward(self, x=None, nodalAtt=None, seq_lengths=None, umask=None, no_cuda=True):
#         if nodalAtt:
#             att_emotions, _ = attentive_node_features(x, seq_lengths, umask, self.matchatt)

#             # Reshape att_emotions to have a 2D shape (batch_size, input_dim)
#             att_emotions = att_emotions.view(att_emotions.size(0), -1)
            
#         x = F.relu(self.linear1(x))
#         x = self.dropout(x)
#         x = self.linear2(x)
#         return x

part a test case (uncomment the top part to see the result)

In [40]:
# seq_lengths = [14]  # Sequence length matches the input tensor
# umask = torch.tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])
# inputs = torch.randn(14, 64)  # Example inputs, shape [seq_len, input_dim]

# # Initialize MatchingAttention
# matchatt_layer = MatchingAttention(mem_dim=inputs.shape[1], cand_dim=inputs.shape[1], alpha_dim=1, att_type='general2')

# # Call attentive_node_features
# att_emotions, alpha_list = attentive_node_features(inputs, seq_lengths, umask, matchatt_layer)

# # Print shapes for verification
# print("att_emotions shape:", att_emotions.shape)  # Should be (seq_len, mem_dim)
# print("alpha_list shape:", len(alpha_list))

In [41]:
seq_lengths = [14]
umask = torch.tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])
inputs = torch.randn(14, 64)  # Example inputs, shape [seq_len, input_dim]

model = MyNetwork(input_dim=inputs.shape[1], hidden_dims=[128, 30], output_dim=7, dropout_rate=0.5)
outputs = model(inputs, True, seq_lengths, umask)
print(outputs.shape)  #.Expected torch.Size([14, 7])

torch.Size([14, 7])


part b test case (uncomment the part and the function to see the result)

In [42]:
model = FCClassifier(input_dim=inputs.shape[1], hidden_dim=64, output_dim=7, dropout_rate=0.5)
outputs = model(inputs, True, seq_lengths, umask)
print(outputs.shape)  # Expected torch.Size([14, 7])

torch.Size([14, 7])


end of sample experiment

Actual run

In [43]:
selected_combination

[0]

In [44]:
# for item in rangesTrain:
#     if item[0] == item[1]:
#         print(item[0], item[1])

In [45]:
dataset = FeatureEngineeredDataset(trainList, testList, valList)
dataLoader = DataLoader(dataset, batch_size=1, shuffle=False)

file_path = "data/dump/" + dataset_path + "/BERT_data_for_classifier/results/classifier_test_no_tuning_Df.pkl"
checkFile = os.path.isfile(file_path)

if checkFile: 
    with open(file_path, "rb") as file:
        df_results_sorted = pickle.load(file)
else:
    results = []
    num_epochs = 30
    i = 0
    for trainSet, testSet, valSet in dataLoader:
        if isinstance(trainSet, list):
            trainSet = trainSet[0].squeeze(0)
            testSet = testSet[0].squeeze(0)
            valSet = valSet[0].squeeze(0)
        else:
            trainSet = trainSet.squeeze(0)
            testSet = testSet.squeeze(0)
            valSet = valSet.squeeze(0)

        X_tensor = to_tensor(trainSet)
        Y_tensor = to_tensor(y_train)
        
        print("Curerntly at", dictKey[i])
#         print(X_tensor.shape, Y_tensor.shape)
#         model1, _,  = model_train1(X_set=X_tensor, Y_set=Y_tensor, num_epochs=num_epochs, 
#                                  umask=train_umask, nodalAtt=nodalAttn[i],
#                                seq_len=train_seq_lengths, ranges=rangesTrain)

#         result1 = classify_emotions(model=model1, X_set=X_tensor, Y_set=Y_tensor,  typeSet='train', 
#                                    isSimpleFC=False, i_dict=i, 
#                                    nodalAtt=nodalAttn[i],  umask=train_umask, seq_len=train_seq_lengths,
#                                    ranges=rangesTrain )
#         results.append(result1)
        if i in selected_combination:
            model2, _,  = model_train2(X_set=X_tensor, Y_set=Y_tensor, num_epochs=num_epochs, 
                                     umask=train_umask, nodalAtt=nodalAttn[i],
                                   seq_len=train_seq_lengths, ranges=rangesTrain)

            X_tensor = to_tensor(testSet)
            Y_tensor = to_tensor(y_test)
            result2 = classify_emotions(model=model2, X_set=X_tensor, Y_set=Y_tensor,  typeSet='test', 
                                       isSimpleFC=True, i_dict=i, 
                                       nodalAtt=nodalAttn[i],  umask=test_umask, seq_len=test_seq_lengths,
                                       ranges=rangesTest)
    #         result2 = classify_emotions(model=model2, X_set=X_tensor, Y_set=Y_tensor,  typeSet='train', 
    #                                    isSimpleFC=True, i_dict=i, 
    #                                    nodalAtt=nodalAttn[i],  umask=train_umask, seq_len=train_seq_lengths,
    #                                    ranges=rangesTrain)
            results.append(result2)
        else:
            print("This is skipped")
        i = i+1

    columns = ['data_combination', 'typeSet', 'isSimpleFC', 'Accuracy', 'Recall', 'Weighted-F1', 'F1-micro', 'F1-macro']
    df_results = pd.DataFrame(results, columns=columns)
    df_results_sorted = df_results.sort_values(by='Weighted-F1', ascending=False)

    with open(file_path, 'wb') as file:
        pickle.dump(df_results_sorted, file)

Curerntly at bert


Training stopped early at epoch 6.
Loss difference (0.0005825146748408838) is below the threshold (0.001).
{'anger': {'precision': 0.5714285714285714, 'recall': 0.2868217054263566, 'f1-score': 0.38193548387096776, 'support': 516.0}, 'disgust': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 99.0}, 'fear': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 60.0}, 'joy': {'precision': 0.4820717131474104, 'recall': 0.4888888888888889, 'f1-score': 0.485456369107322, 'support': 495.0}, 'neutral': {'precision': 0.6516804888694893, 'recall': 0.9244582043343653, 'f1-score': 0.7644649257552484, 'support': 1615.0}, 'sadness': {'precision': 0.7, 'recall': 0.026615969581749048, 'f1-score': 0.05128205128205128, 'support': 263.0}, 'surprise': {'precision': 0.5266272189349113, 'recall': 0.5056818181818182, 'f1-score': 0.5159420289855072, 'support': 352.0}, 'accuracy': 0.6082352941176471, 'macro avg': {'precision': 0.4188297131971975, 'recall': 0.31892379805902543, 'f1-score':

In [46]:
df_results_sorted

,data_combination,typeSet,isSimpleFC,Accuracy,Recall,Weighted-F1,F1-micro,F1-macro
0,bert,test,True,0.608235,0.608235,0.549144,0.608235,0.314154


-----------------------------------------------------------------------

<h4> Select top 10 unique data combinations then tune

In [46]:
# top_10_combinations

In [47]:
max_iterations = 9
counter = 0
combination1 = []
combination2 = []
seen_combinations = set()

for idx, row in df_results_sorted.iterrows():
    if counter >= max_iterations:
        break

    if row['data_combination'] in seen_combinations:
        continue

    if row['isSimpleFC']:
        combination2.append(row['data_combination'])
    else:
        combination1.append(row['data_combination'])

    seen_combinations.add(row['data_combination'])
    counter += 1

# Display the results
print("Combination 1 (isSimpleFC=False):", combination1)
print("Combination 2 (isSimpleFC=True):", combination2)

Combination 1 (isSimpleFC=False): []
Combination 2 (isSimpleFC=True): ['bert', 'egat']


In [48]:
indices1 = [key for key, value in dictKey.items() if value in combination1]
indices2 = [key for key, value in dictKey.items() if value in combination2]

print("Indices for isSimpleFC=False:", indices1)
print("Indices for isSimpleFC=True:", indices2)

Indices for isSimpleFC=False: []
Indices for isSimpleFC=True: [0, 14]


In [49]:
selectedTrainDeepList = [trainList[i] for i in indices1]
selectedTestDeepList = [testList[i] for i in indices1]
selectedValDeepList = [valList[i] for i in indices1]

len(selectedTrainDeepList)

0

In [50]:
indices1

[]

In [51]:
# for trainSet in selectedTrainList:
#     print(type(trainSet))

In [52]:
selectedTrainList = [trainList[i] for i in indices2]
selectedTestList = [testList[i] for i in indices2]
selectedValList = [valList[i] for i in indices2]

len(selectedTrainList)

2

<h4> Tuning using random parameters

In [53]:
# it should call both model_train1 and 2

def objective_func(X_train, X_test, X_val, y_train, y_test, y_val, hyperparams, i_dict, isSimpleFC,
                  train_umask, test_umask, val_umask,
                  train_seq_lengths, test_seq_lengths, val_seq_lengths,
                  rangesTrain, rangesTest, rangesDev, nodalAtt):
    results = []
    hyperparams_string = (
        f'num_epochs={hyperparams["num_epochs"]} '
        f'loss_difference_threshold={hyperparams["loss_difference_threshold"]} '
        f'hidden_dims={hyperparams["hidden_dims"]} '
        f'dropout_rate={hyperparams["dropout_rate"]} '
        f'learning_rate={hyperparams["learning_rate"]} '
        f'optimizers={hyperparams["optimizers"]} '
        f'criteria={hyperparams["criteria"]}'
    )    
    print(hyperparams_string)
            
    X_train_tensor = to_tensor(X_train)
    y_train_tensor = to_tensor(y_train).long()
    X_val_tensor = to_tensor(X_val)
    y_val_tensor = to_tensor(y_val).long()
    X_test_tensor = to_tensor(X_test)
    y_test_tensor = to_tensor(y_test).long()
# train
    start_time = time.time()
    if isSimpleFC:
        model, num_epoch = model_train2(X_set=X_train_tensor, Y_set=y_train_tensor, 
                            num_epochs=hyperparams["num_epochs"], loss_difference_threshold=hyperparams["loss_difference_threshold"], 
                            hidden_dims=hyperparams["hidden_dims"], dropout_rate=hyperparams["dropout_rate"],
                            lr=hyperparams["learning_rate"], optimizer_class=hyperparams["optimizers"], 
                            criterion_class=hyperparams["criteria"],
                            umask=train_umask, seq_len=train_seq_lengths, ranges=rangesTrain, nodalAtt=nodalAtt)        
#     else:
#         model, num_epoch = model_train1(X_train_tensor, y_train_tensor, hyperparams["num_epochs"],
#                             hyperparams["batch_size"], hyperparams["loss_difference_threshold"], 
#                             hyperparams["hidden_dims"], hyperparams["dropout_rate"],
#                             hyperparams["learning_rate"], hyperparams["optimizers"], hyperparams["criteria"])
    end_time = time.time()
    elapsed_time = end_time - start_time
# val
    result = classify_emotions(model=model, X_set=X_val_tensor, Y_set=y_val_tensor, typeSet='validation', 
                               isSimpleFC=isSimpleFC, i_dict=i_dict,
                               nodalAtt=nodalAtt, umask=val_umask, seq_len=val_seq_lengths, ranges=rangesDev)
    elapsed_time = time.time() - start_time
    
    result = list(result)
# test

    hyperparams_string = f'num_epochs={hyperparams["num_epochs"]}-loss_difference_threshold={hyperparams["loss_difference_threshold"]}-hidden_dims={hyperparams["hidden_dims"]}-dropout_rate={hyperparams["dropout_rate"]}-learning_rate={hyperparams["learning_rate"]}-optimizers={hyperparams["optimizers"]}-criteria={hyperparams["criteria"]}'
    result.append(elapsed_time)
    result.append(hyperparams_string)
    result.append(num_epoch)
    results.append(result)
# test
    result = classify_emotions(model=model, X_set=X_test_tensor, Y_set=y_test_tensor, typeSet='test', 
                               isSimpleFC=isSimpleFC, i_dict=i_dict,
                               nodalAtt=nodalAtt, umask=test_umask, seq_len=test_seq_lengths, ranges=rangesTest)
    result = list(result)
    result.append(elapsed_time)
    result.append(hyperparams_string)
    result.append(num_epoch)
    results.append(result)
    
#     result = classify_emotions(model, X_test_tensor, y_test_tensor, \
#                                'test', isSimpleFC, i_dict)
    
#     result = list(result)
#     result.append(elapsed_time)
#     result.append(hyperparams_string)
#     result.append(num_epoch)
#     results.append(result)
    
    columns = ['data_combination', 'typeSet', 'isSimpleFC', 'Accuracy', 'Recall', \
               'Weighted-F1', 'F1-micro', 'F1-macro', 'train_time', 'hyperparams', 'num_epoch']
    df = pd.DataFrame(results, columns=columns)
    df_results_sorted = df.sort_values(by='data_combination', ascending=False)
    
    return df_results_sorted


# def objective_func(X_train, X_test, X_val, 
#                y_train, y_test, y_val, hyperparams, i_dict):

In [54]:
def random_search(X_train=None, X_test=None, X_val=None, 
                  y_train=None, y_test=None, y_val=None, 
                  param_grid=None, isSimpleFC=True, i_dict=None,
                  train_umask=None, test_umask=None, val_umask=None,
                  train_seq_lengths=None, test_seq_lengths=None, val_seq_lengths=None,
                  rangesTrain=None, rangesTest=None, rangesDev=None, nodalAtt=False
                 ):
    sub_total_results = pd.DataFrame(columns = ['data_combination', 'typeSet', 'isSimpleFC', 'Accuracy', 'Recall', \
               'Weighted-F1', 'F1-micro', 'F1-macro', 'train_time', 'hyperparams', 'num_epoch'],)
    MAX_EVALS = 5
    for i in range(MAX_EVALS):
        hyperparams = {k: random.sample(v, 1)[0] for k, v in param_grid.items()}

        try:
            new_results = objective_func(X_train, X_test, X_val,  y_train, y_test, y_val, hyperparams, i_dict, isSimpleFC,  
                                        train_umask, test_umask, val_umask,
                                        train_seq_lengths, test_seq_lengths, val_seq_lengths,
                                        rangesTrain, rangesTest, rangesDev, nodalAtt)
            sub_total_results = pd.concat([sub_total_results, new_results], ignore_index=True)
            
        except Exception as e:
            print(f"Error with hyperparams {hyperparams}: {e}")
            continue
    
    # Sort with best score on top
    return sub_total_results 

In [55]:
param_grid1 = {
    'num_epochs': [20, 30, 40],
    'loss_difference_threshold': [0.001, 0.0001],
    'hidden_dims': [[256, 128], [128, 64], [64, 32]],
    'dropout_rate': [0.3, 0.5, 0.7],
    'learning_rate': [0.001, 0.0001, 0.00001],
    'optimizers': [optim.Adam, optim.SGD],
    'criteria': [nn.CrossEntropyLoss, nn.NLLLoss]
}
param_grid2 = {
    'num_epochs': [20, 30, 40],
    'loss_difference_threshold': [0.001, 0.0001],
    'hidden_dims': [128, 256, 512],
    'dropout_rate': [0.3, 0.5, 0.7],
    'learning_rate': [0.001, 0.0001],
    'optimizers': [optim.Adam, optim.SGD],
    'criteria': [nn.CrossEntropyLoss, nn.NLLLoss]
}

<h5> First find the best hyperparameter combination for the DeepClassifier.

In [56]:
def hyperparamTuning(X_trainSet, X_testSet, X_valSet, y_train, y_test, y_val, isSimpleFC, param_grid, indices,
                    train_umask, test_umask, val_umask,
                    train_seq_lengths, test_seq_lengths, val_seq_lengths,
                    rangesTrain, rangesTest, rangesDev, nodalAttn):
    
    total_results = pd.DataFrame(columns = ['data_combination', 'typeSet', 'isSimpleFC', 'Accuracy', 'Recall', \
               'Weighted-F1', 'F1-micro', 'F1-macro', 'train_time', 'hyperparams', 'num_epoch'],)
    
    for i in indices:
        print("============", dictKey[i], "============")
        X_train = X_trainSet[i]
        X_test = X_testSet[i]
        X_val = X_valSet[i]

        sub_total_results = random_search(X_train, X_test, X_val, y_train, y_test, y_val,
                     param_grid, isSimpleFC, i,
                     train_umask, test_umask, val_umask,
                     train_seq_lengths, test_seq_lengths, val_seq_lengths,
                     rangesTrain, rangesTest, rangesDev, nodalAttn[i])

        total_results = pd.concat([sub_total_results, total_results], ignore_index=True)

    return total_results


Uncomment below next time

In [118]:
# file_path = "data/dump/" + dataset_path + "/BERT_data_for_classifier/results/deep_classifier_tuned_Df.pkl"
# checkFile = os.path.isfile(file_path)

# if checkFile: 
#     with open(file_path, "rb") as file:
#         total_results1_sorted = pickle.load(file)
# else:
#     total_results1 = hyperparamTuning(selectedTrainDeepList, selectedTestDeepList, selectedValDeepList, \
#                                  y_train, y_test, y_val, False, param_grid1, indices1)
    
#     total_results1_sorted = total_results1.sort_values(by='Weighted-F1', ascending=False)
#     with open(file_path, 'wb') as file:
#         pickle.dump(total_results1_sorted, file)

In [119]:
# pd.set_option('display.max_rows', None)  # Show all rows
# pd.set_option('display.max_columns', None)  # Show all columns
# pd.set_option('display.width', None)  # Don't limit the width of the display
# pd.set_option('display.max_colwidth', None)  # Don't truncate column content

# total_results1_sorted

In [120]:
# dataset = FeatureEngineeredDataset(trainList, testList, valList)
# dataLoader = DataLoader(dataset, batch_size=1, shuffle=False)

# file_path = "data/dump/" + dataset_path + "/BERT_data_for_classifier/results/classifier_test_no_tuning_Df.pkl"
# checkFile = os.path.isfile(file_path)

# if checkFile: 
#     with open(file_path, "rb") as file:
#         df_results_sorted = pickle.load(file)
# else:
#     results = []
#     num_epochs = 30
#     i = 0
#     for trainSet, testSet, valSet in dataLoader:
#         if isinstance(trainSet, list):
#             trainSet = trainSet[0].squeeze(0)
#             testSet = testSet[0].squeeze(0)
#             valSet = valSet[0].squeeze(0)
#         else:
#             trainSet = trainSet.squeeze(0)
#             testSet = testSet.squeeze(0)
#             valSet = valSet.squeeze(0)

#         X_tensor = to_tensor(trainSet)
#         Y_tensor = to_tensor(y_train)

#         print("Curerntly at", dictKey[i])
# #         model1, _,  = model_train1(X_set=X_tensor, Y_set=Y_tensor, num_epochs=num_epochs, 
# #                                  umask=train_umask, nodalAtt=nodalAttn[i],
# #                                seq_len=train_seq_lengths, ranges=rangesTrain)

# #         result1 = classify_emotions(model=model1, X_set=X_tensor, Y_set=Y_tensor,  typeSet='train', 
# #                                    isSimpleFC=False, i_dict=i, 
# #                                    nodalAtt=nodalAttn[i],  umask=train_umask, seq_len=train_seq_lengths,
# #                                    ranges=rangesTrain )
# #         results.append(result1)
#         if i in selected_combination:
#             model2, _,  = model_train2(X_set=X_tensor, Y_set=Y_tensor, num_epochs=num_epochs, 
#                                      umask=train_umask, nodalAtt=nodalAttn[i],
#                                    seq_len=train_seq_lengths, ranges=rangesTrain)

#             X_tensor = to_tensor(testSet)
#             Y_tensor = to_tensor(y_test)
#             result2 = classify_emotions(model=model2, X_set=X_tensor, Y_set=Y_tensor,  typeSet='test', 
#                                        isSimpleFC=True, i_dict=i, 
#                                        nodalAtt=nodalAttn[i],  umask=test_umask, seq_len=test_seq_lengths,
#                                        ranges=rangesTest)
#     #         result2 = classify_emotions(model=model2, X_set=X_tensor, Y_set=Y_tensor,  typeSet='train', 
#     #                                    isSimpleFC=True, i_dict=i, 
#     #                                    nodalAtt=nodalAttn[i],  umask=train_umask, seq_len=train_seq_lengths,
#     #                                    ranges=rangesTrain)
#             results.append(result2)
#         else:
#             print("This is skipped")
#         i = i+1

#     columns = ['data_combination', 'typeSet', 'isSimpleFC', 'Accuracy', 'Recall', 'Weighted-F1', 'F1-micro', 'F1-macro']
#     df_results = pd.DataFrame(results, columns=columns)
#     df_results_sorted = df_results.sort_values(by='Weighted-F1', ascending=False)

#     with open(file_path, 'wb') as file:
#         pickle.dump(df_results_sorted, file)

In [62]:
file_path = "data/dump/" + dataset_path + "/BERT_data_for_classifier/results/simple_classifier_tuned_Df.pkl"
checkFile = os.path.isfile(file_path)

if checkFile: 
    with open(file_path, "rb") as file:
        total_results2_sorted = pickle.load(file)
else: 

    total_results2 = hyperparamTuning(trainList, testList, valList, 
                                     y_train, y_test, y_val, True, param_grid2, indices2,
                                     train_umask, test_umask, val_umask,
                                     train_seq_lengths, test_seq_lengths, val_seq_lengths,
                                     rangesTrain, rangesTest, rangesDev, nodalAttn)
    
    total_results2_sorted = total_results2.sort_values(by='Weighted-F1', ascending=False)
    with open(file_path, 'wb') as file:
        pickle.dump(total_results2_sorted, file)

============ bert ============
num_epochs=40 loss_difference_threshold=0.001 hidden_dims=256 dropout_rate=0.3 learning_rate=0.0001 optimizers=<class 'torch.optim.adam.Adam'> criteria=<class 'torch.nn.modules.loss.CrossEntropyLoss'>


Training stopped early at epoch 9.
Loss difference (0.0008873431461738357) is below the threshold (0.001).


C:\Users\edayo\AppData\Local\Temp\ipykernel_2984\1387583240.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  sub_total_results = pd.concat([sub_total_results, new_results], ignore_index=True)


{'anger': {'precision': 0.42613636363636365, 'recall': 0.3605769230769231, 'f1-score': 0.390625, 'support': 208.0}, 'disgust': {'precision': 0.5, 'recall': 0.06451612903225806, 'f1-score': 0.11428571428571428, 'support': 31.0}, 'fear': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 54.0}, 'joy': {'precision': 0.5308056872037915, 'recall': 0.5233644859813084, 'f1-score': 0.5270588235294118, 'support': 214.0}, 'neutral': {'precision': 0.6335329341317365, 'recall': 0.8816666666666667, 'f1-score': 0.7372822299651568, 'support': 600.0}, 'sadness': {'precision': 0.41304347826086957, 'recall': 0.1310344827586207, 'f1-score': 0.19895287958115182, 'support': 145.0}, 'surprise': {'precision': 0.5873015873015873, 'recall': 0.5285714285714286, 'f1-score': 0.556390977443609, 'support': 210.0}, 'accuracy': 0.5800273597811217, 'macro avg': {'precision': 0.44154572150490695, 'recall': 0.35567573086960075, 'f1-score': 0.360656517829292, 'support': 1462.0}, 'weighted avg': {'precision': 0

{'anger': {'precision': 0.45161290322580644, 'recall': 0.33653846153846156, 'f1-score': 0.3856749311294766, 'support': 208.0}, 'disgust': {'precision': 1.0, 'recall': 0.06451612903225806, 'f1-score': 0.12121212121212122, 'support': 31.0}, 'fear': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 54.0}, 'joy': {'precision': 0.502262443438914, 'recall': 0.5186915887850467, 'f1-score': 0.5103448275862069, 'support': 214.0}, 'neutral': {'precision': 0.6457073760580411, 'recall': 0.89, 'f1-score': 0.7484232655921513, 'support': 600.0}, 'sadness': {'precision': 0.48, 'recall': 0.16551724137931034, 'f1-score': 0.24615384615384617, 'support': 145.0}, 'surprise': {'precision': 0.5845410628019324, 'recall': 0.5761904761904761, 'f1-score': 0.580335731414868, 'support': 210.0}, 'accuracy': 0.5896032831737346, 'macro avg': {'precision': 0.5234462550749562, 'recall': 0.3644934138465076, 'f1-score': 0.37030638901266716, 'support': 1462.0}, 'weighted avg': {'precision': 0.5555387794734102,

...

{'anger': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 516.0}, 'disgust': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 99.0}, 'fear': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 60.0}, 'joy': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 495.0}, 'neutral': {'precision': 0.475, 'recall': 1.0, 'f1-score': 0.6440677966101694, 'support': 1615.0}, 'sadness': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 263.0}, 'surprise': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 352.0}, 'accuracy': 0.475, 'macro avg': {'precision': 0.06785714285714285, 'recall': 0.14285714285714285, 'f1-score': 0.0920096852300242, 'support': 3400.0}, 'weighted avg': {'precision': 0.225625, 'recall': 0.475, 'f1-score': 0.30593220338983046, 'support': 3400.0}}
num_epochs=40 loss_difference_threshold=0.0001 hidden_dims=256 dropout_rate=0.7 learning_rate=0.0001 optimizers=<class 'torch.optim.sgd.SGD'> criteria=<class 'torch

Training stopped early at epoch 6.
Loss difference (8.351109295245962e-05) is below the threshold (0.0001).
{'anger': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 208.0}, 'disgust': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 31.0}, 'fear': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 54.0}, 'joy': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 214.0}, 'neutral': {'precision': 0.4103967168262654, 'recall': 1.0, 'f1-score': 0.5819592628516004, 'support': 600.0}, 'sadness': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 145.0}, 'surprise': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 210.0}, 'accuracy': 0.4103967168262654, 'macro avg': {'precision': 0.0586281024037522, 'recall': 0.14285714285714285, 'f1-score': 0.08313703755022862, 'support': 1462.0}, 'weighted avg': {'precision': 0.16842546518177787, 'recall': 0.4103967168262654, 'f1-score': 0.2388341708009304, 'support': 1462.0}}
Classifi

Training stopped early at epoch 3.
Loss difference (0.0006364866464494412) is below the threshold (0.001).
{'anger': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 208.0}, 'disgust': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 31.0}, 'fear': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 54.0}, 'joy': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 214.0}, 'neutral': {'precision': 0.4103967168262654, 'recall': 1.0, 'f1-score': 0.5819592628516004, 'support': 600.0}, 'sadness': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 145.0}, 'surprise': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 210.0}, 'accuracy': 0.4103967168262654, 'macro avg': {'precision': 0.0586281024037522, 'recall': 0.14285714285714285, 'f1-score': 0.08313703755022862, 'support': 1462.0}, 'weighted avg': {'precision': 0.16842546518177787, 'recall': 0.4103967168262654, 'f1-score': 0.2388341708009304, 'support': 1462.0}}
Classifie

{'anger': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 208.0}, 'disgust': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 31.0}, 'fear': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 54.0}, 'joy': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 214.0}, 'neutral': {'precision': 0.4103967168262654, 'recall': 1.0, 'f1-score': 0.5819592628516004, 'support': 600.0}, 'sadness': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 145.0}, 'surprise': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 210.0}, 'accuracy': 0.4103967168262654, 'macro avg': {'precision': 0.0586281024037522, 'recall': 0.14285714285714285, 'f1-score': 0.08313703755022862, 'support': 1462.0}, 'weighted avg': {'precision': 0.16842546518177787, 'recall': 0.4103967168262654, 'f1-score': 0.2388341708009304, 'support': 1462.0}}
Classified:  egat
{'anger': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 516.0}, 'disgust': {'preci

Training stopped early at epoch 3.
Loss difference (0.0008145828856114812) is below the threshold (0.001).
{'anger': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 208.0}, 'disgust': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 31.0}, 'fear': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 54.0}, 'joy': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 214.0}, 'neutral': {'precision': 0.4103967168262654, 'recall': 1.0, 'f1-score': 0.5819592628516004, 'support': 600.0}, 'sadness': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 145.0}, 'surprise': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 210.0}, 'accuracy': 0.4103967168262654, 'macro avg': {'precision': 0.0586281024037522, 'recall': 0.14285714285714285, 'f1-score': 0.08313703755022862, 'support': 1462.0}, 'weighted avg': {'precision': 0.16842546518177787, 'recall': 0.4103967168262654, 'f1-score': 0.2388341708009304, 'support': 1462.0}}
Classifie

In [63]:
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)  # Don't limit the width of the display
pd.set_option('display.max_colwidth', None)  # Don't truncate column content
    
total_results2_sorted

,data_combination,typeSet,isSimpleFC,Accuracy,Recall,Weighted-F1,F1-micro,F1-macro,train_time,hyperparams,num_epoch
11,bert,test,True,0.626471,0.626471,0.588749,0.626471,0.379069,49.790972,num_epochs=40-loss_difference_threshold=0.001-hidden_dims=256-dropout_rate=0.3-learning_rate=0.0001-optimizers=<class 'torch.optim.adam.Adam'>-criteria=<class 'torch.nn.modules.loss.CrossEntropyLoss'>,9
13,bert,test,True,0.622941,0.622941,0.587210,0.622941,0.374739,110.768788,num_epochs=20-loss_difference_threshold=0.0001-hidden_dims=256-dropout_rate=0.7-learning_rate=0.0001-optimizers=<class 'torch.optim.adam.Adam'>-criteria=<class 'torch.nn.modules.loss.CrossEntropyLoss'>,20
17,bert,test,True,0.606471,0.606471,0.554103,0.606471,0.346103,255.620198,num_epochs=30-loss_difference_threshold=0.0001-hidden_dims=256-dropout_rate=0.5-learning_rate=0.001-optimizers=<class 'torch.optim.adam.Adam'>-criteria=<class 'torch.nn.modules.loss.CrossEntropyLoss'>,30
12,bert,validation,True,0.589603,0.589603,0.547065,0.589603,0.370306,110.768788,num_epochs=20-loss_difference_threshold=0.0001-hidden_dims=256-dropout_rate=0.7-learning_rate=0.0001-optimizers=<class 'torch.optim.adam.Adam'>-criteria=<class 'torch.nn.modules.loss.CrossEntropyLoss'>,20
10,bert,validation,True,0.580027,0.580027,0.537376,0.580027,0.360657,49.790972,num_epochs=40-loss_difference_threshold=0.001-hidden_dims=256-dropout_rate=0.3-learning_rate=0.0001-optimizers=<class 'torch.optim.adam.Adam'>-criteria=<class 'torch.nn.modules.loss.CrossEntropyLoss'>,9
19,bert,test,True,0.593529,0.593529,0.527389,0.593529,0.289279,13.496133,num_epochs=40-loss_difference_threshold=0.001-hidden_dims=512-dropout_rate=0.5-learning_rate=0.001-optimizers=<class 'torch.optim.sgd.SGD'>-criteria=<class 'torch.nn.modules.loss.NLLLoss'>,6
16,bert,validation,True,0.560876,0.560876,0.501518,0.560876,0.321394,255.620198,num_epochs=30-loss_difference_threshold=0.0001-hidden_dims=256-dropout_rate=0.5-learning_rate=0.001-optimizers=<class 'torch.optim.adam.Adam'>-criteria=<class 'torch.nn.modules.loss.CrossEntropyLoss'>,30
18,bert,validation,True,0.547196,0.547196,0.470028,0.547196,0.275096,13.496133,num_epochs=40-loss_difference_threshold=0.001-hidden_dims=512-dropout_rate=0.5-learning_rate=0.001-optimizers=<class 'torch.optim.sgd.SGD'>-criteria=<class 'torch.nn.modules.loss.NLLLoss'>,6
7,egat,test,True,0.475000,0.475000,0.305932,0.475000,0.092010,483.034221,num_epochs=20-loss_difference_threshold=0.0001-hidden_dims=128-dropout_rate=0.5-learning_rate=0.001-optimizers=<class 'torch.optim.adam.Adam'>-criteria=<class 'torch.nn.modules.loss.CrossEntropyLoss'>,20
9,egat,test,True,0.475000,0.475000,0.305932,0.475000,0.092010,53.089077,num_epochs=30-loss_difference_threshold=0.001-hidden_dims=128-dropout_rate=0.3-learning_rate=0.0001-optimizers=<class 'torch.optim.adam.Adam'>-criteria=<class 'torch.nn.modules.loss.NLLLoss'>,3


### Use the tuned result to predict the test

In [57]:
hyperparams = {
    'num_epochs': 30,
    'loss_difference_threshold': 0.001,
    'hidden_dims': 512,
    'dropout_rate': 0.3,
    'learning_rate': 0.0001,
    'optimizers': optim.Adam,
    'criteria': nn.NLLLoss
}

tuned_results = pd.DataFrame(columns = ['data_combination', 'typeSet', 'isSimpleFC', 'Accuracy', 'Recall', \
           'Weighted-F1', 'F1-micro', 'F1-macro', 'train_time', 'hyperparams', 'num_epoch'],)

new_results = objective_func(trainList[0], testList[0], valList[0],  y_train, y_test, y_val, hyperparams, 0, True,  
                            train_umask, test_umask, val_umask,
                            train_seq_lengths, test_seq_lengths, val_seq_lengths,
                            rangesTrain, rangesTest, rangesDev, nodalAttn[0])
tuned_results = pd.concat([tuned_results, new_results], ignore_index=True)
tuned_results

num_epochs=30 loss_difference_threshold=0.001 hidden_dims=512 dropout_rate=0.3 learning_rate=0.0001 optimizers=<class 'torch.optim.adam.Adam'> criteria=<class 'torch.nn.modules.loss.NLLLoss'>


Training stopped early at epoch 8.
Loss difference (8.681618849343264e-05) is below the threshold (0.001).
{'anger': {'precision': 0.3755868544600939, 'recall': 0.38461538461538464, 'f1-score': 0.38004750593824227, 'support': 208.0}, 'disgust': {'precision': 0.5, 'recall': 0.06451612903225806, 'f1-score': 0.11428571428571428, 'support': 31.0}, 'fear': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 54.0}, 'joy': {'precision': 0.4830508474576271, 'recall': 0.5327102803738317, 'f1-score': 0.5066666666666667, 'support': 214.0}, 'neutral': {'precision': 0.6385542168674698, 'recall': 0.8833333333333333, 'f1-score': 0.7412587412587412, 'support': 600.0}, 'sadness': {'precision': 0.40384615384615385, 'recall': 0.14482758620689656, 'f1-score': 0.2131979695431472, 'support': 145.0}, 'surprise': {'precision': 0.626984126984127, 'recall': 0.3761904761904762, 'f1-score': 0.47023809523809523, 'support': 210.0}, 'accuracy': 0.5649794801641587, 'macro avg': {'precision': 0.4325745999450

C:\Users\edayo\AppData\Local\Temp\ipykernel_2984\4238940293.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  tuned_results = pd.concat([tuned_results, new_results], ignore_index=True)


,data_combination,typeSet,isSimpleFC,Accuracy,Recall,Weighted-F1,F1-micro,F1-macro,train_time,hyperparams,num_epoch
0,bert,validation,True,0.564979,0.564979,0.523556,0.564979,0.346528,74.815786,num_epochs=30-loss_difference_threshold=0.001-...,8
1,bert,test,True,0.630588,0.630588,0.596563,0.630588,0.387082,74.815786,num_epochs=30-loss_difference_threshold=0.001-...,8


rerun again to correct the model name in the df

In [58]:
hyperparams = {
    'num_epochs': 20,
    'loss_difference_threshold': 0.0001,
    'hidden_dims': 128,
    'dropout_rate': 0.5,
    'learning_rate': 0.0001,
    'optimizers': optim.Adam,
    'criteria': nn.NLLLoss
}

tuned_results = pd.DataFrame(columns = ['data_combination', 'typeSet', 'isSimpleFC', 'Accuracy', 'Recall', \
           'Weighted-F1', 'F1-micro', 'F1-macro', 'train_time', 'hyperparams', 'num_epoch'],)

new_results = objective_func(trainList[3], testList[3], valList[3],  y_train, y_test, y_val, hyperparams, 3, True,  
                            train_umask, test_umask, val_umask,
                            train_seq_lengths, test_seq_lengths, val_seq_lengths,
                            rangesTrain, rangesTest, rangesDev, nodalAttn[3])
tuned_results = pd.concat([tuned_results, new_results], ignore_index=True)
tuned_results

num_epochs=20 loss_difference_threshold=0.0001 hidden_dims=128 dropout_rate=0.5 learning_rate=0.0001 optimizers=<class 'torch.optim.adam.Adam'> criteria=<class 'torch.nn.modules.loss.NLLLoss'>


Training stopped early at epoch 5.
Loss difference (6.66746559242204e-05) is below the threshold (0.0001).
{'anger': {'precision': 0.4178082191780822, 'recall': 0.2932692307692308, 'f1-score': 0.3446327683615819, 'support': 208.0}, 'disgust': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 31.0}, 'fear': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 54.0}, 'joy': {'precision': 0.49767441860465117, 'recall': 0.5, 'f1-score': 0.4988344988344988, 'support': 214.0}, 'neutral': {'precision': 0.6055734190782422, 'recall': 0.9416666666666667, 'f1-score': 0.7371167645140247, 'support': 600.0}, 'sadness': {'precision': 0.5714285714285714, 'recall': 0.05517241379310345, 'f1-score': 0.10062893081761007, 'support': 145.0}, 'surprise': {'precision': 0.564935064935065, 'recall': 0.4142857142857143, 'f1-score': 0.47802197802197804, 'support': 210.0}, 'accuracy': 0.5663474692202463, 'macro avg': {'precision': 0.3796313847463732, 'recall': 0.31491343221638785, 'f1-score': 

C:\Users\edayo\AppData\Local\Temp\ipykernel_2984\439966065.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  tuned_results = pd.concat([tuned_results, new_results], ignore_index=True)


,data_combination,typeSet,isSimpleFC,Accuracy,Recall,Weighted-F1,F1-micro,F1-macro,train_time,hyperparams,num_epoch
0,bert-select-more,validation,True,0.566347,0.566347,0.503201,0.566347,0.308462,16.634499,num_epochs=20-loss_difference_threshold=0.0001...,5
1,bert-select-more,test,True,0.605000,0.605000,0.548043,0.605000,0.312679,16.634499,num_epochs=20-loss_difference_threshold=0.0001...,5


In [59]:
hyperparams = {
    'num_epochs': 40,
    'loss_difference_threshold': 0.0001,
    'hidden_dims': 128,
    'dropout_rate': 0.5,
    'learning_rate': 0.0001,
    'optimizers': optim.Adam,
    'criteria': nn.NLLLoss
}

tuned_results = pd.DataFrame(columns = ['data_combination', 'typeSet', 'isSimpleFC', 'Accuracy', 'Recall', \
           'Weighted-F1', 'F1-micro', 'F1-macro', 'train_time', 'hyperparams', 'num_epoch'],)

new_results = objective_func(trainList[1], testList[1], valList[1],  y_train, y_test, y_val, hyperparams, 1, True,  
                            train_umask, test_umask, val_umask,
                            train_seq_lengths, test_seq_lengths, val_seq_lengths,
                            rangesTrain, rangesTest, rangesDev, nodalAttn[1])
tuned_results = pd.concat([tuned_results, new_results], ignore_index=True)
tuned_results

num_epochs=40 loss_difference_threshold=0.0001 hidden_dims=128 dropout_rate=0.5 learning_rate=0.0001 optimizers=<class 'torch.optim.adam.Adam'> criteria=<class 'torch.nn.modules.loss.NLLLoss'>


Training stopped early at epoch 8.
Loss difference (4.013472982331012e-05) is below the threshold (0.0001).
{'anger': {'precision': 0.4, 'recall': 0.2980769230769231, 'f1-score': 0.3415977961432507, 'support': 208.0}, 'disgust': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 31.0}, 'fear': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 54.0}, 'joy': {'precision': 0.4557522123893805, 'recall': 0.48130841121495327, 'f1-score': 0.4681818181818182, 'support': 214.0}, 'neutral': {'precision': 0.5963003264417845, 'recall': 0.9133333333333333, 'f1-score': 0.7215273206056616, 'support': 600.0}, 'sadness': {'precision': 0.5555555555555556, 'recall': 0.034482758620689655, 'f1-score': 0.06493506493506493, 'support': 145.0}, 'surprise': {'precision': 0.5098039215686274, 'recall': 0.37142857142857144, 'f1-score': 0.4297520661157025, 'support': 210.0}, 'accuracy': 0.5444596443228454, 'macro avg': {'precision': 0.3596302879936212, 'recall': 0.29980428538206727, 'f1-score

C:\Users\edayo\AppData\Local\Temp\ipykernel_2984\3109196931.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  tuned_results = pd.concat([tuned_results, new_results], ignore_index=True)


,data_combination,typeSet,isSimpleFC,Accuracy,Recall,Weighted-F1,F1-micro,F1-macro,train_time,hyperparams,num_epoch
0,bert-select-few,validation,True,0.544460,0.544460,0.481411,0.544460,0.289428,22.454779,num_epochs=40-loss_difference_threshold=0.0001...,8
1,bert-select-few,test,True,0.602353,0.602353,0.545936,0.602353,0.309043,22.454779,num_epochs=40-loss_difference_threshold=0.0001...,8


In [60]:
hyperparams = {
    'num_epochs': 40,
    'loss_difference_threshold': 0.001,
    'hidden_dims': 512,
    'dropout_rate': 0.7,
    'learning_rate': 0.001,
    'optimizers': optim.Adam,
    'criteria': nn.NLLLoss
}

tuned_results = pd.DataFrame(columns = ['data_combination', 'typeSet', 'isSimpleFC', 'Accuracy', 'Recall', \
           'Weighted-F1', 'F1-micro', 'F1-macro', 'train_time', 'hyperparams', 'num_epoch'],)

new_results = objective_func(trainList[2], testList[2], valList[2],  y_train, y_test, y_val, hyperparams, 2, True,  
                            train_umask, test_umask, val_umask,
                            train_seq_lengths, test_seq_lengths, val_seq_lengths,
                            rangesTrain, rangesTest, rangesDev, nodalAttn[2])
tuned_results = pd.concat([tuned_results, new_results], ignore_index=True)
tuned_results

num_epochs=40 loss_difference_threshold=0.001 hidden_dims=512 dropout_rate=0.7 learning_rate=0.001 optimizers=<class 'torch.optim.adam.Adam'> criteria=<class 'torch.nn.modules.loss.NLLLoss'>


Training stopped early at epoch 4.
Loss difference (0.0003139507074802761) is below the threshold (0.001).
{'anger': {'precision': 0.3950617283950617, 'recall': 0.3076923076923077, 'f1-score': 0.34594594594594597, 'support': 208.0}, 'disgust': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 31.0}, 'fear': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 54.0}, 'joy': {'precision': 0.5428571428571428, 'recall': 0.4439252336448598, 'f1-score': 0.4884318766066838, 'support': 214.0}, 'neutral': {'precision': 0.5991140642303433, 'recall': 0.9016666666666666, 'f1-score': 0.7198935462408517, 'support': 600.0}, 'sadness': {'precision': 0.4666666666666667, 'recall': 0.04827586206896552, 'f1-score': 0.0875, 'support': 145.0}, 'surprise': {'precision': 0.5242718446601942, 'recall': 0.5142857142857142, 'f1-score': 0.5192307692307693, 'support': 210.0}, 'accuracy': 0.5574555403556771, 'macro avg': {'precision': 0.36113877811562983, 'recall': 0.316549397765502, 'f1-score':

C:\Users\edayo\AppData\Local\Temp\ipykernel_2984\1846461806.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  tuned_results = pd.concat([tuned_results, new_results], ignore_index=True)


,data_combination,typeSet,isSimpleFC,Accuracy,Recall,Weighted-F1,F1-micro,F1-macro,train_time,hyperparams,num_epoch
0,bert-select-mod,validation,True,0.557456,0.557456,0.499414,0.557456,0.308715,19.460354,num_epochs=40-loss_difference_threshold=0.001-...,4
1,bert-select-mod,test,True,0.609412,0.609412,0.561854,0.609412,0.342603,19.460354,num_epochs=40-loss_difference_threshold=0.001-...,4
